# ページの種類を判定

- 今回はDBPediaに「出身地」等の情報があるページを取得している
- それが「人物」であるかは自明ではない（音楽グループ等などが含まれる）
- それを選別可能か？

**=> 結論: 構造化された情報が整備されていないできない**

---

In [ ]:
import json

from SPARQLWrapper import SPARQLWrapper

In [ ]:
sparql = SPARQLWrapper(endpoint="http://ja.dbpedia.org/sparql", returnFormat="json")

## 「人物」か


- DBPedia
  - [About: 大谷翔平](https://ja.dbpedia.org/page/%E5%A4%A7%E8%B0%B7%E7%BF%94%E5%B9%B3) => `An Entity of Type : Athlete`
  - [About: 加藤一二三](https://ja.dbpedia.org/page/%E5%8A%A0%E8%97%A4%E4%B8%80%E4%BA%8C%E4%B8%89) => `An Entity of Type : Thing`
  - [About: サカナクション](https://ja.dbpedia.org/page/%E3%82%B5%E3%82%AB%E3%83%8A%E3%82%AF%E3%82%B7%E3%83%A7%E3%83%B3) => `An Entity of Type : MusicalArtist`

In [ ]:
PERSON_PROPERTIES = {
    "http://xmlns.com/foaf/0.1/Person",
    "http://dbpedia.org/ontology/Person",
    "http://schema.org/Person"
}

def is_person(page_title):
    sparql.setQuery(f"""
        SELECT DISTINCT * 
        WHERE {{
            <http://ja.dbpedia.org/resource/{page_title}>
            <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>
            ?o .
            }}
    """)
    
    res = sparql.query().convert()
    for x in res["results"]["bindings"]:
        if x["o"]["value"] in PERSON_PROPERTIES:
            return True
    
    return False

In [ ]:
is_person("大谷翔平")

In [ ]:
is_person("北海道")

In [ ]:
is_person("加藤一二三")

In [ ]:
is_person("サカナクション")

---

In [ ]:
def is_natural_person(page):
    sparql.setQuery(f"""
        SELECT DISTINCT * 
        WHERE {{
            <http://ja.dbpedia.org/resource/{page}>
            rdf:type
            ?o .
            }}
    """)

    res = sparql.query().convert()
    rdf_types = [x["o"]["value"] for x in res["results"]["bindings"]]

    return "http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson" in rdf_types

In [ ]:
is_natural_person("大谷翔平")

In [ ]:
is_natural_person("北海道")

In [ ]:
is_natural_person("加藤一二三")

In [ ]:
is_natural_person("サカナクション")

## 音楽グループか

In [ ]:
def is_music_group(page):
    sparql.setQuery(f"""
        SELECT DISTINCT * 
        WHERE {{
            <http://ja.dbpedia.org/resource/{page}>
            rdf:type
            ?o .
            }}
    """)

    res = sparql.query().convert()
    rdf_types = [x["o"]["value"] for x in res["results"]["bindings"]]

    return "http://schema.org/MusicGroup" in rdf_types

In [ ]:
is_music_group("大谷翔平")

In [ ]:
is_music_group("加藤一二三")

In [ ]:
is_music_group("北海道")

In [ ]:
is_music_group("GLAY")

In [ ]:
is_music_group("米津玄師")

In [ ]:
is_music_group("あいみょん")